In [ ]:
from keras.layers.core import Activation, Dense, Dropout, SpatialDropout1D
from keras.layers import Embedding
from keras.layers import LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os


In [ ]:
ftrain = open("C:\\Users\\sree\\Downloads\\lstmz\\LSTM\\training.txt",encoding='UTF8')
print(ftrain)


<_io.TextIOWrapper name='C:\\Users\\sree\\Downloads\\lstmz\\LSTM\\training.txt' mode='r' encoding='UTF8'>


In [ ]:
maxline=0
word_freq=collections.Counter()
num_recs=0
print(len(word_freq))


0


In [ ]:
for line in ftrain:
    label,sentense=line.strip().split("\t")
    words=nltk.word_tokenize(sentense.lower())
    if len(words)>maxline:
        maxline=len(words)
    for word in words:
        word_freq[word]+=1
    num_recs+=1
print(maxline)
print(num_recs)
print(word_freq)


42
7086
Counter({'i': 4707, ',': 4194, '.': 3364, 'the': 3221, 'and': 2150, '!': 2131, 'potter': 2093, 'harry': 2088, 'vinci': 2001, 'da': 1998, 'brokeback': 1996, 'code': 1994, 'mountain': 1994, 'love': 1542, 'is': 1521, '...': 1441, 'a': 1305, 'was': 1179, '..': 1153, 'awesome': 1126, 'impossible': 1091, 'mission': 1090, 'like': 974, 'it': 899, 'to': 808, 'movie': 783, 'that': 718, "'s": 629, 'because': 608, 'sucks': 601, 'sucked': 596, 'hate': 578, 'so': 506, 'as': 473, 'my': 462, 'much': 452, '``': 449, 'of': 424, 'really': 374, 'movies': 366, 'stupid': 365, 'you': 330, 'down': 322, 'but': 295, 'we': 295, 'be': 290, 'with': 288, 'just': 287, 'one': 283, 'know': 276, 'suck': 276, '3': 271, 'out': 269, 'who': 263, '/': 263, 'or': 263, 'am': 259, 'loved': 256, 'want': 254, 'into': 253, 'which': 251, 'right': 249, 'for': 241, 'an': 225, "n't": 224, 'this': 213, 'me': 213, ':': 207, 'are': 203, 'think': 202, 'not': 198, 'how': 197, 'if': 195, '....': 190, 'depressing': 188, 'people': 18

In [ ]:
MAX_features=2000
MAX_sentense_len=40

In [ ]:
vocab_size=min(MAX_features,len(word_freq))+2
print(vocab_size)

2002


In [ ]:
word2index={x[0]: i+2 for i,x in enumerate (word_freq.most_common(MAX_features))}
print(word2index)

{'i': 2, ',': 3, '.': 4, 'the': 5, 'and': 6, '!': 7, 'potter': 8, 'harry': 9, 'vinci': 10, 'da': 11, 'brokeback': 12, 'code': 13, 'mountain': 14, 'love': 15, 'is': 16, '...': 17, 'a': 18, 'was': 19, '..': 20, 'awesome': 21, 'impossible': 22, 'mission': 23, 'like': 24, 'it': 25, 'to': 26, 'movie': 27, 'that': 28, "'s": 29, 'because': 30, 'sucks': 31, 'sucked': 32, 'hate': 33, 'so': 34, 'as': 35, 'my': 36, 'much': 37, '``': 38, 'of': 39, 'really': 40, 'movies': 41, 'stupid': 42, 'you': 43, 'down': 44, 'but': 45, 'we': 46, 'be': 47, 'with': 48, 'just': 49, 'one': 50, 'know': 51, 'suck': 52, '3': 53, 'out': 54, 'who': 55, '/': 56, 'or': 57, 'am': 58, 'loved': 59, 'want': 60, 'into': 61, 'which': 62, 'right': 63, 'for': 64, 'an': 65, "n't": 66, 'this': 67, 'me': 68, ':': 69, 'are': 70, 'think': 71, 'not': 72, 'how': 73, 'if': 74, '....': 75, 'depressing': 76, 'people': 77, 'his': 78, 'would': 79, 'horrible': 80, 'up': 81, 'reading': 82, 'why': 83, 'series': 84, 'there': 85, 'in': 86, 'only'

In [ ]:
word2index['PAD']=0
word2index['UNK']=1

In [ ]:
print(word2index)

{'i': 2, ',': 3, '.': 4, 'the': 5, 'and': 6, '!': 7, 'potter': 8, 'harry': 9, 'vinci': 10, 'da': 11, 'brokeback': 12, 'code': 13, 'mountain': 14, 'love': 15, 'is': 16, '...': 17, 'a': 18, 'was': 19, '..': 20, 'awesome': 21, 'impossible': 22, 'mission': 23, 'like': 24, 'it': 25, 'to': 26, 'movie': 27, 'that': 28, "'s": 29, 'because': 30, 'sucks': 31, 'sucked': 32, 'hate': 33, 'so': 34, 'as': 35, 'my': 36, 'much': 37, '``': 38, 'of': 39, 'really': 40, 'movies': 41, 'stupid': 42, 'you': 43, 'down': 44, 'but': 45, 'we': 46, 'be': 47, 'with': 48, 'just': 49, 'one': 50, 'know': 51, 'suck': 52, '3': 53, 'out': 54, 'who': 55, '/': 56, 'or': 57, 'am': 58, 'loved': 59, 'want': 60, 'into': 61, 'which': 62, 'right': 63, 'for': 64, 'an': 65, "n't": 66, 'this': 67, 'me': 68, ':': 69, 'are': 70, 'think': 71, 'not': 72, 'how': 73, 'if': 74, '....': 75, 'depressing': 76, 'people': 77, 'his': 78, 'would': 79, 'horrible': 80, 'up': 81, 'reading': 82, 'why': 83, 'series': 84, 'there': 85, 'in': 86, 'only'

In [ ]:
print(len(word2index))

2002


In [ ]:
index2word={v:k for k,v in word2index.items()}
index2word

{2: 'i',
 3: ',',
 4: '.',
 5: 'the',
 6: 'and',
 7: '!',
 8: 'potter',
 9: 'harry',
 10: 'vinci',
 11: 'da',
 12: 'brokeback',
 13: 'code',
 14: 'mountain',
 15: 'love',
 16: 'is',
 17: '...',
 18: 'a',
 19: 'was',
 20: '..',
 21: 'awesome',
 22: 'impossible',
 23: 'mission',
 24: 'like',
 25: 'it',
 26: 'to',
 27: 'movie',
 28: 'that',
 29: "'s",
 30: 'because',
 31: 'sucks',
 32: 'sucked',
 33: 'hate',
 34: 'so',
 35: 'as',
 36: 'my',
 37: 'much',
 38: '``',
 39: 'of',
 40: 'really',
 41: 'movies',
 42: 'stupid',
 43: 'you',
 44: 'down',
 45: 'but',
 46: 'we',
 47: 'be',
 48: 'with',
 49: 'just',
 50: 'one',
 51: 'know',
 52: 'suck',
 53: '3',
 54: 'out',
 55: 'who',
 56: '/',
 57: 'or',
 58: 'am',
 59: 'loved',
 60: 'want',
 61: 'into',
 62: 'which',
 63: 'right',
 64: 'for',
 65: 'an',
 66: "n't",
 67: 'this',
 68: 'me',
 69: ':',
 70: 'are',
 71: 'think',
 72: 'not',
 73: 'how',
 74: 'if',
 75: '....',
 76: 'depressing',
 77: 'people',
 78: 'his',
 79: 'would',
 80: 'horrible',
 

In [ ]:
x=np.empty((num_recs,),dtype=list)
x


array([None, None, None, ..., None, None, None], dtype=object)

In [ ]:
print(x.shape)

(7086,)


In [ ]:
y=np.zeros(num_recs,)
y

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
ftrain = open("C:\\Users\\sree\\Downloads\\lstmz\\LSTM\\training.txt",encoding='UTF8')
i=0
for line in ftrain:
    label,sentense=line.strip().split("\t")
    words=nltk.word_tokenize(sentense.lower())
    seqs=[]
#     print((words))
    for word in words:
        if word in word2index:
            seqs.append(word2index[word])
        else:
            seqs.append(word2index['UNK'])
    x[i]=seqs
    y[i]=int(label)
    i+=1

ftrain.close()
print(x)


[list([5, 11, 10, 13, 101, 16, 49, 21, 4])
 list([67, 19, 5, 116, 960, 961, 2, 354, 136, 111, 3, 45, 316, 315, 24, 962, 3, 6, 11, 10, 13, 137, 117, 963, 340, 67, 4])
 list([2, 123, 5, 11, 10, 13, 18, 322, 4]) ...
 list([35, 2, 300, 97, 3, 157, 5, 301, 27, 220, 3, 2, 58, 302, 39, 73, 37, 2, 303, 5, 27, 12, 14, 4])
 list([95, 12, 14, 16, 144, 18, 80, 27, 4])
 list([91, 3, 6, 12, 14, 19, 18, 89, 27, 4])]


In [ ]:
print(y)

[1. 1. 1. ... 0. 0. 0.]


In [ ]:

from keras_preprocessing.sequence import pad_sequences
X=pad_sequences(x,maxlen=MAX_sentense_len)
print(X)
print(X.shape)



[[  0   0   0 ...  49  21   4]
 [  0   0   0 ... 340  67   4]
 [  0   0   0 ...  18 322   4]
 ...
 [  0   0   0 ...  12  14   4]
 [  0   0   0 ...  80  27   4]
 [  0   0   0 ...  89  27   4]]
(7086, 40)


In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# embeding layer


In [ ]:
EMBEDDING_SIZE = 128
HIDDEN_LAYER_SIZE = 64
BATCH_SIZE = 32
NUM_EPOCHS = 10


In [ ]:
model=Sequential()
model.add(Embedding(vocab_size,EMBEDDING_SIZE,input_length=MAX_sentense_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(HIDDEN_LAYER_SIZE,dropout=0.3,recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer ='adam', metrics =['accuracy'])


In [ ]:
model.fit(Xtrain, ytrain, batch_size =BATCH_SIZE, epochs =NUM_EPOCHS, validation_data=(Xtest, ytest))


Epoch 1/10
178/178 [==============================] - 7s 28ms/step - loss: 0.2210 - accuracy: 0.8993 - val_loss: 0.0536 - val_accuracy: 0.9803
Epoch 2/10
178/178 [==============================] - 4s 23ms/step - loss: 0.0244 - accuracy: 0.9917 - val_loss: 0.0467 - val_accuracy: 0.9852
Epoch 3/10
178/178 [==============================] - 5s 26ms/step - loss: 0.0061 - accuracy: 0.9988 - val_loss: 0.1081 - val_accuracy: 0.9767
Epoch 4/10
178/178 [==============================] - 5s 26ms/step - loss: 0.0082 - accuracy: 0.9975 - val_loss: 0.0425 - val_accuracy: 0.9852
Epoch 5/10
178/178 [==============================] - 4s 25ms/step - loss: 0.0027 - accuracy: 0.9996 - val_loss: 0.0481 - val_accuracy: 0.9901
Epoch 6/10
178/178 [==============================] - 5s 26ms/step - loss: 0.0022 - accuracy: 0.9996 - val_loss: 0.0470 - val_accuracy: 0.9894
Epoch 7/10
178/178 [==============================] - 4s 25ms/step - loss: 0.0017 - accuracy: 0.9996 - val_loss: 0.0517 - val_accuracy: 0.9887

In [ ]:
loss, acc = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)


45/45 [==============================] - 0s 5ms/step - loss: 0.0595 - accuracy: 0.9873


In [ ]:
print('Test loss: %.3f, accuracy: %.3f'% (loss, acc))


Test loss: 0.059, accuracy: 0.987


In [ ]:
for i in range(5):
    idx = np.random.randint(len(Xtest))  #randomly taking some index
    xtest = Xtest[idx].reshape(1, 40)
    print(xtest)
    ylabel = ytest[idx]
    ypred = model.predict(xtest)[0][0]
    sent = ' ' .join([index2word[x] for x in xtest[0].tolist() if x != 0] )
    print(' %.5f %d %s'% (int(ypred), ylabel, sent))


[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  5 11 10 13 16 21 20]]
1/1 [==============================] - 0s 14ms/step
 0.00000 1 the da vinci code is awesome ..
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  5 11 10 13 16 21  7  7]]
1/1 [==============================] - 0s 14ms/step
 0.00000 1 the da vinci code is awesome ! !
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  2 15  9  8  4]]
1/1 [==============================] - 0s 14ms/step
 0.00000 1 i love harry potter .
[[  0   0   0   0   9   8 280 281 282 165 186 283  44 223  78 284   6  32
  178  61  78 285  48 286   3 161 287 288   6 289   6 290 177   5 291 292
    3 293 180  17]]
1/1 [==============================] - 0s 14ms/step
 0.00000 0 harry potter dragged draco malfoy ’ s trousers down past his hips and sucked him into his throat with vigor , makin